In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

# matplotlib configuration
sns.set_style(style='ticks')
sns.set_theme(style="ticks")
sns.set(rc={'figure.figsize':(11.7,8.27)})

# pandas configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [2]:
raw_df = pd.read_csv('Airbnb_Open_Data.csv')
raw_df.rename(columns = dict(zip(raw_df.columns, [a.replace(' ', '_') for a in raw_df.columns])),inplace=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host_id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host_name                       102193 non-null  object 
 5   neighbourhood_group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   instant_bookable                102494 non-null  object 
 10  cancellation_policy             102523 non-null  object 
 11  room_type                       102599 non-null  object 
 12  Construction_yea

In [4]:
raw_df.isna().sum()

id                                     0
NAME                                 250
host_id                                0
host_identity_verified               289
host_name                            406
neighbourhood_group                   29
neighbourhood                         16
lat                                    8
long                                   8
instant_bookable                     105
cancellation_policy                   76
room_type                              0
Construction_year                    214
price                                247
service_fee                          273
minimum_nights                       409
number_of_reviews                    183
last_review                        15893
reviews_per_month                  15879
review_rate_number                   326
calculated_host_listings_count       319
availability_365                     448
house_rules                        52131
license                           102597
dtype: int64

In [5]:
# defining 
directory_dict = {}
for a, b in raw_df.groupby(['neighbourhood_group', 'neighbourhood']).count().index:
    directory_dict[b] = a

In [6]:
raw_df[(raw_df.neighbourhood_group.isna()) | (raw_df.neighbourhood.isna())][['neighbourhood', 'neighbourhood_group']]

,neighbourhood,neighbourhood_group
74,Washington Heights,NaN
75,Clinton Hill,NaN
76,East Village,NaN
77,Upper East Side,NaN
78,Woodside,NaN
90,Williamsburg,NaN
91,Bushwick,NaN
92,Prospect Heights,NaN
148,East Village,NaN
161,Williamsburg,NaN


#### ====================================================================================================

In [7]:
def tweak_df(df):
    return (df
            .rename(
                columns = dict(zip(df.columns, [a.replace(' ', '_') for a in df.columns]))
            )
            .assign(
                NAME = df.NAME.fillna('Unfilled Entry'),
                host_identity_verified = df.host_identity_verified.fillna('unconfirmed'),
                host_name = df.host_name.fillna('unfilled'),
                last_review = pd.to_datetime(df.last_review, format='%m/%d/%Y', errors='coerce')
            )
            .astype({
                'lat': 'float32',
                'long': 'float32',
                'host_identity_verified': 'category',
                'instant_bookable': 'category',
                'cancellation_policy': 'category',
                'room_type': 'category',
                'neighbourhood_group': 'category'
            })
    ).drop(
        columns = 'license',
#         index = df[df.lat.isna()].index + df[df.long.isna()].index
    )

# original memory usage = 106568597
tweak_df(raw_df).head()
# tweak_df(raw_df).memory_usage(deep=True).sum()

,id,NAME,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,instant_bookable,cancellation_policy,room_type,Construction_year,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.647491,-73.972366,False,strict,Private room,2020.0,$966,$193,10.0,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.753620,-73.983772,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,unconfirmed,Elise,Manhattan,Harlem,40.809021,-73.941902,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,1002755,Unfilled Entry,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.685139,-73.959763,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,2019-07-05,4.64,4.0,1.0,322.0,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.798512,-73.943993,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."


In [9]:
tweak_df(raw_df).isna().sum()

id                                    0
NAME                                  0
host_id                               0
host_identity_verified                0
host_name                             0
neighbourhood_group                  29
neighbourhood                        16
lat                                   8
long                                  8
instant_bookable                    105
cancellation_policy                  76
room_type                             0
Construction_year                   214
price                               247
service_fee                         273
minimum_nights                      409
number_of_reviews                   183
last_review                       15893
reviews_per_month                 15879
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    448
house_rules                       52131
dtype: int64

In [8]:
for each in raw_df.columns:
    print(each, ' : ' , raw_df[each].nunique())

id  :  102058
NAME  :  61281
host_id  :  102057
host_identity_verified  :  2
host_name  :  13190
neighbourhood_group  :  7
neighbourhood  :  224
lat  :  21991
long  :  17774
instant_bookable  :  2
cancellation_policy  :  3
room_type  :  4
Construction_year  :  20
price  :  1151
service_fee  :  231
minimum_nights  :  153
number_of_reviews  :  476
last_review  :  2477
reviews_per_month  :  1016
review_rate_number  :  5
calculated_host_listings_count  :  78
availability_365  :  438
house_rules  :  1975
license  :  1
